In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%writefile .env

TAVILY_API_KEY = your_key
JINA_API_KEY = your_key
CHROME_INSTANCE_PATH = /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
BROWSER_HEADLESS=False


Overwriting .env


In [3]:
import sys
from src.workflow import run_agent_workflow

In [4]:
user_query = '''
    I would like to analyze Amazon product sales data.
    The target for analysis is the './data/Amazon_Sale_Report.csv' file.
    Please conduct an analysis to extract marketing insights based on this data.
    Please perform various analytical techniques starting from basic data attribute exploration, product sales trends, variable relationships, variable combinations, etc.
    If there are any additional analyses needed to extract insights after the data analysis, please perform those as well.
    Please include detailed analysis in the report along with supporting images and charts.
    Please save the final report in PDF format.
'''

user_query = '''
    이것은 아마존 상품판매 데이터를 분석하고 싶습니다.
    분석대상은 "./data/Amazon_Sale_Report.csv" 파일 입니다.
    데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요.
    분석은 기본적인 데이터 속성 탐색 부터, 상품 판매 트렌드, 변수 관계, 변수 조합 등 다양한 분석 기법을 수행해 주세요.
    데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요.
    분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요.
    최종 리포트는 pdf 형태로 저장해 주세요.
'''

user_query = '''
    이것은 아마존 상품판매 데이터를 분석하고 싶습니다.
    분석대상은 "./data/Dat-fresh-food-claude.csv" 파일 입니다.
    데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요.
    분석은 기본적인 데이터 속성 탐색 부터, 상품 판매 트렌드, 변수 관계, 변수 조합 등 다양한 분석 기법을 수행해 주세요.
    데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요.
    분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요.
    최종 리포트는 pdf 형태로 저장해 주세요.
'''

#user_query = "안녕 나는 장동진이야"
#user_query = '''
#    이것은 아마존 상품판매 데이터를 분석하고 싶습니다. 분석대상은 "./data/Amazon_Sale_Report.csv" 파일 입니다. 데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요. 분석은 간단하게 1개정도만 해 주세요. 코드 테스트 중이거든요 데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요. 분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요. 최종 리포트는 pdf 형태로 저장해 주세요.
#    '''

# user_query = '''
#     영문으로 작성된 pptx 파일을 한글로 변환하고 싶습니다.
#     대상은 "./data/bedrock.pptx" 파일 입니다.
#     슬라이드의 영문을 한글로 번역하고 슬라이드 structure는 유지해 주세요.
# '''

# user_query = '''
#     주어진 프로젝트 소스코드는 "Bedrock-Manus: AI automation framework optimized for Amazon Bedrock and business use cases 입니다."
#     프로젝트 소스코드는 "./data/08_bedrock_manus.tar.gz" 파일 입니다.
#     나는 Streamlit을 이용해서 이 소스코드에 대한 UI를 만들고 싶습니다. 
#     제가 원하는 요구사항은
#     1. 현재 스텝이 무엇인지 (coodinator, supervisor, planner 등등) 보이는 곳이 있으면 좋겠습니다.
#         - 스텝에 대한 자세한 정보는 "./README.md"를 참고하시면 됩니다. 
#     2. 각 스텝에서 진행되는 것이 보였으면 좋겠습니다. 즉, 각 스텝에서 llm이 생성하는 토큰들을 실시간으로 볼 수 있으면 좋겠습니다. 
#     3. "app.py" 파일만 생성하지 말고, 필요하다면 기존의 다른 파일 (workflow.py 등)들도  수정하세요. 
#     4. 수정된 파일은 모두 ./artifacts 에 넣어주세요.
#     참고할 사항이 있습니다. 
#     1. 소스코드 수행은 "main.py"로 수행됩니다. 
#     2. "./src/service/workflow_service.py" 이 파일은 참고하지 마세요. 

#     UI 생성을 위한 프로젝트 코드를 작성해주세요
#     결과물은 ./artifacts 에 넣어주세요.
# '''

# user_query = '''
#     "2025년 중국 가전 신제품 출시에 대한 리포트 작성해줘."
# '''

In [5]:
import os
import shutil

def remove_artifact_folder(folder_path="./artifacts/"):
    """
    ./artifact/ 폴더가 존재하면 삭제하는 함수
    
    Args:
        folder_path (str): 삭제할 폴더 경로
    """
    if os.path.exists(folder_path):
        print(f"'{folder_path}' 폴더를 삭제합니다...")
        try:
            # 폴더와 그 내용을 모두 삭제
            shutil.rmtree(folder_path)
            print(f"'{folder_path}' 폴더가 성공적으로 삭제되었습니다.")
        except Exception as e:
            print(f"오류 발생: {e}")
    else:
        print(f"'{folder_path}' 폴더가 존재하지 않습니다.")


import nest_asyncio
nest_asyncio.apply()

remove_artifact_folder()

result = run_agent_workflow(
    user_input=user_query,
    debug=False
)


#슈퍼바이저 에서 코더, 트랙커, 리포터 툴로 만들기 
#코더부터 툴(에이전트 에즈 어 툴)로 만들고 
#트랙커는 새로 만들기


# Print the conversation history
print("\n=== Conversation History ===")
print ("result", result)
for history in result["history"]:

    print ("===")
    print (f'agent: {history["agent"]}')
    print (f'message: {history["message"]}')


INFO [src.workflow] ===== Starting workflow =====

INFO [src.workflow] 
user input: 
    이것은 아마존 상품판매 데이터를 분석하고 싶습니다.
    분석대상은 "./data/Dat-fresh-food-claude.csv" 파일 입니다.
    데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요.
    분석은 기본적인 데이터 속성 탐색 부터, 상품 판매 트렌드, 변수 관계, 변수 조합 등 다양한 분석 기법을 수행해 주세요.
    데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요.
    분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요.
    최종 리포트는 pdf 형태로 저장해 주세요.


INFO [src.graph.nodes] ===== Coordinator talking...... =====

INFO [src.utils.strands_sdk_utils] COORDINATOR - Prompt Cache Disabled


'./artifacts/' 폴더를 삭제합니다...
'./artifacts/' 폴더가 성공적으로 삭제되었습니다.
handoff_to_planner: 안녕하세요, Bedrock-Manus입니다. 데이터 분석과 리포트 생성에 관한 복잡한 요청이므로, 이 작업을 플래너 시스템에 전달하여 체계적으로 처리하도록 하겠습니다. 플래너가 데이터 탐색부터 인사이트 도출, PDF 리포트 생성까지 단계별로 수행할 수 있도록 하겠습니다.


INFO [src.graph.nodes] ===== Coordinator updated shared global state with 5 keys =====

INFO [src.graph.nodes] ===== Coordinator completed task =====

INFO [src.graph.nodes] ===== Planner generating plan =====

INFO [src.graph.nodes] 
Planner received task:
[ {'text': 'Original Task:'},
  'request',
  'request_prompt',
  {'text': '\nInputs from previous nodes:'},
  {'text': '\nFrom coordinator:'},
  { 'text': '  - Agent: handoff_to_planner: 안녕하세요, Bedrock-Manus입니다. 데이터 분석과 리포트 생성에 관한 복잡한 요청이므로, '
            '이 작업을 플래너 시스템에 전달하여 체계적으로 처리하도록 하겠습니다. 플래너가 데이터 탐색부터 인사이트 도출, PDF 리포트 생성까지 단계별로 수행할 수 '
            '있도록 하겠습니다.\n'}]

INFO [src.graph.nodes] 
Planner received kwargs:
{}

INFO [src.graph.nodes] ===== Successfully retrieved shared state from global storage =====

INFO [src.graph.nodes] 
Shared state:
{ 'goto': 'planner',
  'history': [ { 'agent': 'coordinator',
                 'message': 'handoff_to_planner: 안녕하세요, Bedrock-Manus입니다. 데이터 분석과 리포트 생성에 관한 복잡한 '
                      

사용자의 요청을 분석해보겠습니다:

1. 데이터 파일: "./data/Dat-fresh-food-claude.csv"
2. 분석 목적: 아마존 상품판매 데이터를 기반으로 마케팅 인사이트 추출
3. 분석 요구사항:
   - 기본적인 데이터 속성 탐색
   - 상품 판매 트렌드 분석
   - 변수 간 관계 분석
   - 변수 조합 분석
   - 추가 인사이트를 위한 심화 분석
4. 결과물: 상세 분석과 시각화 자료가 포함된 PDF 형태의 리포트

이 작업을 수행하기 위해 다음과 같은 계획이 필요합니다:

1. Coder: 데이터 로드, 탐색적 데이터 분석(EDA), 통계 분석, 시각화 등 모든 데이터 관련 작업을 수행합니다.
   - 데이터 로드 및 기본 속성 확인
   - 결측치, 이상치 처리 및 데이터 전처리
   - 기술통계량 분석
   - 시계열 분석을 통한 판매 트렌드 파악
   - 상관관계 분석을 통한 변수 간 관계 파악
   - 변수 조합을 통한 세그먼트별 분석
   - 인사이트 도출을 위한 추가 분석
   - 분석 결과에 대한 시각화 작업

2. Reporter: 코더가 분석한 결과를 바탕으로 최종 리포트를 작성합니다.
   - 분석 목적 및 배경 작성
   - 데이터 개요 요약
   - 주요 분석 결과 정리
   - 마케팅 인사이트 도출 및 제안
   - 시각화 자료 포함한 PDF 리포트 생성

이제 이 계획을 정리하여 실행 가능한 형태로 만들겠습니다.# Plan

## thought
사용자는 아마존 상품판매 데이터("./data/Dat-fresh-food-claude.csv")를 분석하고 마케팅 인사이트를 도출하기를 원합니다. 이를 위해 기본적인 데이터 탐색부터 판매 트렌드, 변수 관계 분석까지 수행하고 시각화 자료와 함께 PDF 리포트로 제공해야 합니다.

## title:
아마존 신선식품 판매 데이터 분석 및 마케팅 인사이트 도출

## steps:
### 1. Coder: 데이터 분석 및 시각화
- [ ] 데이터셋("./data/Dat-fr


INFO [src.graph.nodes] ===== Planner analyzing and creating execution plan =====

INFO [src.graph.nodes] ===== Planner completed plan generation =====

INFO [src.graph.nodes] ===== Supervisor evaluating next action =====

INFO [src.graph.nodes] 
Supervisor received task:
[ {'text': 'Original Task:'},
  'request',
  'request_prompt',
  {'text': '\nInputs from previous nodes:'},
  {'text': '\nFrom planner:'},
  { 'text': '  - Agent: # Plan\n'
            '\n'
            '## thought\n'
            '사용자는 아마존 상품판매 데이터("./data/Dat-fresh-food-claude.csv")를 분석하고 마케팅 인사이트를 도출하기를 원합니다. 이를 '
            '위해 기본적인 데이터 탐색부터 판매 트렌드, 변수 관계 분석까지 수행하고 시각화 자료와 함께 PDF 리포트로 제공해야 합니다.\n'
            '\n'
            '## title:\n'
            '아마존 신선식품 판매 데이터 분석 및 마케팅 인사이트 도출\n'
            '\n'
            '## steps:\n'
            '### 1. Coder: 데이터 분석 및 시각화\n'
            '- [ ] 데이터셋("./data/Dat-fresh-food-claude.csv") 로드 및 구조 파악\n'
            '- [ ] 결측치, 이상치 처리 및 기본 데이터 전처리\n'
            '- [ ] 기술통계량

...

Based on the response from the reporter agent and the clues provided, I need to update the task completion status for the plan. Let me analyze what tasks have been completed by the Reporter:

### Reporter's completed tasks from the response:
1. **Analysis purpose and background introduction**: The reporter created a comprehensive report that includes an executive summary and analysis background.
2. **Dataset overview summary**: The report includes a dataset overview, mentioning "1,220건의 주문 데이터 로드 및 구조 파악" (loading and understanding the structure of 1,220 order records).
3. **Summary of key analysis results (including visualization)**: The report contains key analysis results with visualizations, mentioning "15개의 차트와 시각화 자료 생성" (creation of 15 charts and visualization materials).
4. **Derivation and explanation of marketing insights**: The report includes sections on customer segment insights, product and category insights, and promotion and pricing strategy insights.
5. **Presentation 


INFO [tracker_agent_tool] ===== Updated full_plan with tracking results =====

INFO [tracker_agent_tool] ===== Updated shared state with tracking results =====

INFO [tracker_agent_tool] ===== Tracker Agent Tool completed successfully =====


tracker result_text # Plan

## thought
  - 데이터 분석 작업과 리포트 작성이 모두 성공적으로 완료되었습니다. Coder가 모든 계획된 분석 항목을 수행했고, Reporter가 최종 분석 리포트를 작성하여 PDF로 변환하였습니다. 모든 결과물이 성공적으로 생성되어 프로젝트가 완료되었습니다.

## title:
  - 아마존 신선식품 판매 데이터 분석 및 마케팅 인사이트 도출

## steps:
### 1. Coder: 데이터 분석 및 시각화
- [x] 데이터셋("./data/Dat-fresh-food-claude.csv") 로드 및 구조 파악
- [x] 결측치, 이상치 처리 및 기본 데이터 전처리
- [x] 기술통계량 분석 및 데이터 분포 확인
- [x] 시간에 따른 판매 트렌드 분석 (시계열 분석)
- [x] 상품 카테고리별 성과 분석
- [x] 변수 간 상관관계 분석
- [x] 고객 세그먼트별 구매 패턴 분석
- [x] 가격 민감도 분석
- [x] 인사이트 도출을 위한 추가 분석 (예: RFM 분석, 장바구니 분석)
- [x] 모든 분석 결과에 대한 시각화 작업 (그래프, 차트 생성)
- [x] 분석 결과 저장 및 주요 인사이트 요약

### 2. Reporter: 최종 분석 리포트 작성 및 PDF 생성
- [x] 분석 목적 및 배경 소개
- [x] 데이터셋 개요 요약
- [x] 주요 분석 결과 정리 (시각화 자료 포함)
- [x] 마케팅 인사이트 도출 및 설명
- [x] 액션 아이템 및 전략적 제안 제시
- [x] 전체 내용을 체계적인 PDF 리포트로 생성 및 저장
# Amazon Fresh Food Sales Data Analysis - Project Completed Successfully! 📊

Great news! All tasks in your Amazon Fresh Food sales data analysis project have been successfully completed. Here's a summary

ERROR:strands.multiagent.graph:node_id=<supervisor>, error=<Expecting value: line 1 column 1 (char 0)> | node failed
ERROR:strands.multiagent.graph:graph execution failed
Traceback (most recent call last):
  File "/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_agentcore/setup/.venv/lib/python3.12/site-packages/strands/multiagent/graph.py", line 309, in invoke_async
    await self._execute_graph()
  File "/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_agentcore/setup/.venv/lib/python3.12/site-packages/strands/multiagent/graph.py", line 349, in _execute_graph
    await task
  File "/home/ubuntu/.local/share/uv/python/cpython-3.12.11-linux-aarch64-gnu/lib/python3.12/asyncio/futures.py", line 289, in __await__
    yield self  # This tells Task to wait for completion.
    ^^^^^^^^^^
  File "/home/ubuntu/.loc

JSONDecodeError: Expecting value: line 1 column 1 (char 0)